In [1]:
import os
import re
import pandas as pd

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
travel_time_dir = ['D:/Travel Time Prediction/truck 2019 data']
save_path = 'D:/Travel Time Prediction/truck 2019 3hr split'

In [4]:
def init_save_directory(save_path):
    if not os.path.exists(os.path.join(save_path)):
        os.mkdir(save_path)
    if not os.path.exists(os.path.join(save_path, 'start_stop')):
        os.mkdir(save_path + '/start_stop/')
    if not os.path.exists(os.path.join(save_path, 'stop_start')):
        os.mkdir(save_path + '/stop_start/')

    return [save_path + '/start_stop/', save_path + '/stop_start/']

def create_time_series(travel_time_dir, save_path):
    file_error_path_list = []

    # init directory for save time series
    save_dir = init_save_directory(save_path)
    
    for idx in range(2):
        for road_dir in os.listdir(travel_time_dir[idx]):
            file_names = os.listdir(travel_time_dir[idx] + f"/{road_dir}/")
            df_list = []
            for file_name in file_names:
                datetime = re.split("_|[.]", file_name)[:3]
                try:
                    df_travel = pd.read_csv(travel_time_dir[idx] + f"/{road_dir}/" + file_name)
                except:
                    file_error_path_list.append(f"{travel_time_dir[idx]}/{road_dir}/{file_name}\n")
                    continue
                _tmp = {
                    'datetime': (pd.to_datetime(f"{datetime[2]}-{datetime[0]}-{datetime[1]}") + pd.to_timedelta(list(set(df_travel['time_range'])), unit='h')).to_list(),
                    'delta_t': df_travel[['time_range', 'delta_t']].groupby(['time_range']).mean()['delta_t'].to_list(),
                    'day_of_week': pd.to_datetime(f"{datetime[2]}-{datetime[0]}-{datetime[1]}").dayofweek,
                }
                df_list.append(pd.DataFrame(_tmp))

                # remove data in memory
                del df_travel
                del _tmp

            df = pd.concat(df_list, axis='rows', ignore_index=True)
            df.to_csv(os.path.join(save_dir[idx], f"{road_dir}.csv"), index=False)

            # remove data in memory
            del df_list
            del df

    print(f"create time series complete.")
    print(f"save list of file error(no data in file) in file error.txt")
    # save error list to file
    f = open('E:/data/error.txt', 'w')
    f.writelines(file_error_path_list)
        

In [28]:
pd.to_timedelta(3, unit='h')

Timedelta('0 days 03:00:00')

In [11]:
def create_time_series_split_3hr(travel_time_dir, save_path):
    file_error_path_list = []

    # init directory for save time series
    save_dir = init_save_directory(save_path)
    
    for idx in range(1):
        for road_dir in os.listdir(travel_time_dir[idx]):
            file_names = os.listdir(travel_time_dir[idx] + f"/{road_dir}/")
            df_list = []
            for file_name in file_names:
                datetime = re.split("_|[.]", file_name)[:3]
                try:
                    df_travel = pd.read_csv(travel_time_dir[idx] + f"/{road_dir}/" + file_name)
                except:
                    file_error_path_list.append(f"{travel_time_dir[idx]}/{road_dir}/{file_name}\n")
                    continue
                dt = []
                delta_t = []
                dow = []
                num_truck = []
                standard = []
                for i in range(0,21,3):

                   
                    dt.append((pd.to_datetime(f"{datetime[2]}-{datetime[0]}-{datetime[1]}") + pd.to_timedelta(i, unit='h')))
                    delta_t.append(df_travel[(df_travel["time_range"] >= i) & (df_travel["time_range"] < i+3) ]["delta_t"].mean())
                    dow.append(pd.to_datetime(f"{datetime[2]}-{datetime[0]}-{datetime[1]}").day_name()  )
                    num_truck.append(len(df_travel[(df_travel["time_range"] >= i) & (df_travel["time_range"] < i+3) ]))
                    standard.append(df_travel[(df_travel["time_range"] >= i) & (df_travel["time_range"] < i+3) ]["delta_t"].std())

                _tmp = {
                    'datetime': dt,
                    'delta_t': delta_t,  
                    'day_of_week': dow,
                    'number_of_truck' : num_truck,
                    'std': standard}
                df_list.append(pd.DataFrame(_tmp))

                # remove data in memory
                del df_travel
                del _tmp

            df = pd.concat(df_list, axis='rows', ignore_index=True)
            df.to_csv(os.path.join(save_dir[idx], f"{road_dir}.csv"), index=False)

            # remove data in memory
            del df_list
            del df

    print(f"create time series complete.")
    print(f"save list of file error(no data in file) in file error.txt")
    # save error list to file
    f = open('D:/Travel Time Prediction/error3.txt', 'w')
    f.writelines(file_error_path_list)
        

In [12]:
create_time_series_split_3hr(travel_time_dir, save_path)

create time series complete.
save list of file error(no data in file) in file error.txt
